# Week 3 Notebook 3: Data Cleaning & Wrangling

In this lesson, we are going to practice our data cleaning and wrangling skills. 

We will learn how to:
- Reshape data by adding and removing columns
- Handle missing values by dropping rows and columns
- Handle missing values by imputing values
- Convert data types
- Split data
- Create simple data visualisations

We will use the same `jobs` data set as the previous lesson (from [Pavan Tinniru on Kaggle](https://www.kaggle.com/pavantanniru/-datacleaningforbeginnerusingpandas)).

First, we need to load the data set.

In [ ]:
import pandas as pd

jobs = pd.read_csv('Data-cleaning-for-beginners-using-pandas.csv')

Take a look at our data set again. There are not so many rows, so let's `print` to show all our data.

In [ ]:
print(jobs)

We can check the data types using `dtypes`.

In [ ]:
jobs.dtypes

### Dropping Columns

The first thing we want to do is to remove the redundant column `Index` from our DataFrame, because Pandas has automatically provided an index for the rows. We will save the result in a new `DataFrame` object.

In [ ]:
# Using 'drop' method to drop columns
jobs_2 = jobs.drop(columns = 'Index')
jobs_2.head()

The original DataFrame `jobs` is not modified by the above command.

We can also use the parameter `inplace=True` which will modify the existing DataFrame `jobs`, because the changes will be made "in-place" which means the same DataFrame.

In [ ]:
# Modify the DataFrame in-place
jobs.drop(columns = 'Index', inplace = True)
jobs.head()

### Missing Values

The original data set also contains missing values. One way to check the number of missing values is to combine two methods `isnull()` and `sum()`.

- `isnull()` returns `True` if a value is null and `False` otherwise.
- `sum()` adds up the `True` values. `True` evaluates to 1 and `False` evaluates to 0

In [ ]:
jobs.isnull().sum()

We can see that this gives us 7 null values for the `Age` column and 1 null value for the `Rating` column. 

We can drop rows or columns that contain missing values using `dropna()`. 

In [ ]:
# Drop rows that contain any null values
jobs_dropByRow = jobs.dropna()
print(jobs_dropByRow)

In [ ]:
# Use the argument 'axis=1' to drop columns that contains any null values
# (by default the argument is 'axis=0', which means drop by row)
jobs_dropByCol = jobs.dropna(axis=1)
print(jobs_dropByCol)

As you can see, this will just remove the missing values completely, but we might also lose important data from the other rows and columns!

### Missing Value Imputation

A better strategy to treat missing values would be to **impute** it with new values. Usually, this would be an "average" value or the most probable value for the particular column.

For a column with numerical values, the imputation method is usually the *mean* or *median*. 

For a column with categorical values, the imputation method is usually the category that occurs most frequently.

For example, we could impute the mean age from the data set into any missing `Age` values. 

In [ ]:
# Impute by calculating mean into missing values using 'fillna()''
jobs['Age'].fillna(jobs['Age'].mean(), inplace = True)

In [ ]:
print(jobs)

As you can see, this maintains the total number of rows as 29. However, some of the `Age` values now have the mean value 39.045455. This may not be very meaningful.

### Data Type Conversion

We can convert the data types of a column using the `astype()` method. For example, let's convert the `Age` column from `float` into `int`. This will just remove the decimal places from the floating-point values as they are converted into integers.

In [ ]:
# Replace the values in the 'Age column with the values converted to int'
jobs['Age'] = jobs['Age'].astype('int')

# Check the results
jobs.head()

### Replacing Values

Another useful Pandas method is `replace()`, which will replace existing values with new ones to make sure that the data is consistent. For example, the values in `Location` are not very consistent.

In [ ]:
# Count the number of each value
jobs['Location'].value_counts()

Some of the locations are recorded with a comma and some without. It also seems redundant to store an acronym of the location, such as 'New York, Ny' and 'India, In'. Let's use the `replace()` method.

Regular Expressions or Regex are used to match strings. The regular expression `'India.*'` means a string that starts with the letters 'India' followed by any character, any number of times. So, this would help to match `India,In` and `India In`. We are using the argument `regex=True` to indicate that the letters to be replaced are defined using regular expressions.

You can read more about [Regular Expressions](https://www.computerhope.com/jargon/r/regex.htm) from Computer Hope. 

In [ ]:
# Replace any location starting with the letters 'New York' with just 'New York',
# any location starting with the letters 'India' with 'India'
# any location starting with the letters 'Australia' with 'Australia'.
jobs['Location'].replace('India.*', 'India', regex = True, inplace = True)
jobs['Location'].replace('New York.*', 'New York', regex = True, inplace = True)
jobs['Location'].replace('Australia.*', 'Australia', regex = True, inplace = True)

### Replacing with Lists

We can also notice that the values in `Easy Apply` are either 'TRUE' or '-1'. Let's check by counting each value.

In [ ]:
jobs['Easy Apply'].value_counts()

We can use the `replace()` method to replace the value '-1' with the Boolean value `False` and the string value 'TRUE' with the Boolean value `True`. We can pass both values to be replaced and the new values as List into the method.

In [ ]:
# Replace values in 'Easy Apply' column
# -1 should be replaced with False and 'TRUE' should be replaced with True.
jobs['Easy Apply'].replace(['-1','TRUE'],[False, True], inplace = True)
jobs.head()

Check the data type now to ensure the column stores Boolean (bool) values.

In [ ]:
jobs.dtypes

### Replacing with Null Values

We can also replace values with `NaN`, which means 'Not a Number', null, or missing value. Although we dropped or imputed missing values earlier, sometimes it might be better to keep the value as missing rather than an invalid value.

For example, there are several values in the `Rating` column and `Established` column which are -1. Keeping the value as -1 might cause calculations or plotting to be not meaningful.

We can use the `numpy` library to obtain the `NaN` value as `numpy` is usually used for numerical calculations. 

In [ ]:
import numpy as np

jobs['Rating'].replace(-1,np.NaN, inplace = True )
jobs['Established'].replace(-1, np.NaN, inplace = True)
print(jobs)

### Splitting Data

Another common data wrangling task would be to split data into more than one column, especially if there has been columns that records multiple string.

For example, the `Salary` column contains a string with a lower and upper bound for salary values. However, it might be more useful if the salary was a numeric value so that we could calculate the *mean* or *median* salary.

We can use the string `split()` function for this by specifying *where* we want to split the string.
The `expand` argument also indicates that we want to expand the values into more than one column.

Now, let's split the `Salary` column into two new columns.

In [ ]:
# Split the 'Salary' and add two new columns
jobs[['Min Salary', 'Max Salary']] = jobs['Salary'].str.split('-', n = 1, expand = True)
jobs.head()

Check the data type of the new columns.

In [ ]:
jobs.dtypes

Since `Min Salary` and `Max Salary` are of type `object`, let's convert them into `integer` values. We can use the `astype()` method, but before that let's strip the 'k' and '$' characters from the right and left sides respectively.

In [ ]:
# Remove the special characters from 'Min Salary' and convert it to an integer
jobs['Min Salary'] = jobs['Min Salary'].str.rstrip('k')
jobs['Min Salary'] = jobs['Min Salary'].str.lstrip('$')
jobs['Min Salary'] = jobs['Min Salary'].astype(int)

# Do the same for 'Max Salary'
jobs['Max Salary'] = jobs['Max Salary'].str.rstrip('k')
jobs['Max Salary'] = jobs['Max Salary'].str.lstrip('$')
jobs['Max Salary'] = jobs['Max Salary'].astype(int)


In [ ]:
jobs.head()

Now, we can calculate the estimated salary to be the midpoint of the `Min Salary` and `Max Salary`.

In [ ]:
# Calculate an estimated salary for each job as a new column
jobs['Est Salary'] = (jobs['Max Salary'] + jobs['Min Salary'])/2
jobs.head()

## Plotting with Pandas

Now that we have cleaned up the data, we can perform some simple visualisation/plotting with Pandas.

For example, let's create a pie chart using `plot.pie`.

In [ ]:
# Obtain the value counts for each location
jobs1 = jobs['Location'].value_counts()
print(jobs1)
print(jobs1.index)

Now that we have counts of the locations, we can plot the values and use the index as the labels.

In [ ]:
# Plot pie chart
jobs1.plot.pie(labels = jobs1.index, autopct = '%.1f%%', 
               colors = ["lightpink", "violet", "lightblue"])

### Bar Charts

A bar chart can be created in Pandas by grouping the categorical data.

For example, we want to find the mean estimated salary by location and whether it is 'Easy Apply'.

In [ ]:
# First, find the mean salary grouped by 'Location', then 'Easy Apply' values
meanSalary = jobs.groupby(['Location', 'Easy Apply'])['Est Salary'].mean()
meanSalary

This gives us the mean salary grouped by location, then whether it is Easy Apply. 

We can unstack the group so that there the rows are by Location, but 'Easy Apply' values are arranged in columns. This returns a DataFrame object. 

In [ ]:
# unstack so that the Easy Apply Values are arranged in columns
meanSalaryByLocation = meanSalary.unstack()
meanSalaryByLocation

Now, we can create a simple bar plot to show the mean salaries, which are the values in each row.

In [ ]:
# Plot a bar chart 
meanSalaryByLocation.plot.bar(ylabel='Mean Estimated Salary')

### Stacked Bar Chart

Let's try to create a stacked bar chart to show how many jobs are being counted. We can use the same `groupby`, but this time we use the `count()` function to count the number of rows.

In [ ]:
# Count rows by Location and Easy Apply
countLocation = jobs.groupby(['Location', 'Easy Apply'])['Est Salary'].count()
countLocation

Similarly, we can unstack it.

In [ ]:
countByLocation = countLocation.unstack()
countByLocation

We can stack the values so that it shows the total count of jobs by location.

In [ ]:
# Create a stacked bar chart
countByLocation.plot.bar(stacked = True, ylabel = 'Number of Jobs')

### Scatter Plot

Now that we have the 'Estimated Salary' as a numeric value, we can also create a quick scatter plot with Pandas.

In [ ]:
# Create a scatter plot of Estimated Salary vs Rating
jobs.plot.scatter(x = 'Rating', y = 'Est Salary', marker = '*', s = 50)

## Exercises

The Bike Sharing Data Set contains data about bike rentals from Capital Bikeshare on the 1st of December 2021. The data is adapted from [Capital Bike Share's System Data](https://www.capitalbikeshare.com/system-data).

In [ ]:
# Loading in the data set into a DataFrame called 'bikes'
bikes = pd.read_csv("bikeshare.csv")
bikes.head()

**Q1 Check Data Types**

Check the data types for each of the columns of the `bikes` data set.

In [ ]:
# Q1 Answer


**Q2 Dropping Columns**

Drop the columns `ride_id`, `start_station_name` and `end_station_name` and save the result in a new object called `newbikes`.

In [ ]:
# Q2 Answer


**Q3 Count Missing Values**

Check the number of missing values for each column in the `bikes` data set.

In [ ]:
# Q3 Answer


**Q4 Drop Missing Values**

Since there are over 7000 rows of data, drop all the rows that contain any null values by using the `inplace` parameter to update the data set.

In [ ]:
# Q4 Answer


**Q5 Convert to DateTime Type**

The starting time of the bike rental is interpreted by Python as a string. We should convert it using the Pandas method `to_datetime()` as follows:<br>
`bikes['started_at'] = pd.to_datetime(bikes['started_at'])`

Write the statement above and also convert the `ended_at` column to a `datetime` data type. Then run `bikes.dtypes` again to check the data types.

In [ ]:
# Q5 Answer


**Q6 Add Column**

Add a column called `duration`, which is the difference between the values in the `ended_at` and `started_at` columns, then check the data types again.

In [ ]:
# Q6 Answer



**Q7 Descriptive Statistics **

The duration is a `timedelta` object. Use `describe` to view information about `bikes['duration']`.

In [ ]:
# Q7 Answer


The time delta is given in terms of days, hours, minutes and seconds.
We can convert the `timedelta` into seconds using the `DateTime` method `total_seconds()` like this: <br>
`bikes['duration'].dt.total_seconds()`.

By using this method, we can add a new column called `duration_in_min` that calculates the duration in ***minutes***.

In [ ]:
# Adding a new column which is the duration in MINUTES
bikes['duration_in_min'] =bikes['duration'].dt.total_seconds() / 60

Let's check the value counts of the `rideable_type` column.

In [ ]:
bikes['rideable_type'].value_counts()

**Q8 Replacing Values**

By using the `replace()` method, modify the 'rideable_type' column by changing all values of `classic_bike` into `classic`, `electric_bike` into `electric` and `docked_bike` into `docked`. 

In [ ]:
# Q8 Answer



In [ ]:
bikes.head()

**Q9 Group By**

Group the bikes by `rideable_type`, then `member_casual` and **count** the number of `ride_id` in each group.
Store the results in a new object called `bikes_group`.

In [ ]:
# Q9 Solution



**Q10 Unstacking Group**

Unstack the `bikes_group` group so that the result is a DataFrame of `rideable_type` with columns `casual` and `member`.

In [ ]:
# Q10 Answer


**Q11 Plot**

Create a bar plot using the DataFrame from Q10, with `stacked=True` to create a stacked bar chart.

In [ ]:
# Q11 Answer

